In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [18]:
# Carga y manipulación inicial de los datos
df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')
df = df.drop(columns=['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'CAEC', 'SCC', 'FAF'])

#Se separa la acoluma a predecir en el modelo de machine learning y también se vuelve binaria la variable de respuesta
y = df["NObeyesdad"]
df = df.drop(columns=['NObeyesdad'])

In [19]:
## Preparación de los datos para el modelo de machine learning

#Conversión de variables categóricas binarias a unos y ceros

df['SMOKE'] = df['SMOKE'].map({'yes': 1, 'no': 0})
df['FAVC'] = df['FAVC'].map({'yes': 1, 'no': 0})

catcols = df[['FAVC','SMOKE','CALC','MTRANS']]
numcols = df[['FCVC','NCP','CH2O','TUE']]


In [20]:
print(catcols)

      FAVC  SMOKE        CALC                 MTRANS
0        0      0          no  Public_Transportation
1        0      1   Sometimes  Public_Transportation
2        0      0  Frequently  Public_Transportation
3        0      0  Frequently                Walking
4        0      0   Sometimes  Public_Transportation
...    ...    ...         ...                    ...
2106     1      0   Sometimes  Public_Transportation
2107     1      0   Sometimes  Public_Transportation
2108     1      0   Sometimes  Public_Transportation
2109     1      0   Sometimes  Public_Transportation
2110     1      0   Sometimes  Public_Transportation

[2111 rows x 4 columns]


In [21]:
# Inicializando los preprocesadores
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
mmscaler = MinMaxScaler()

In [22]:
# Ajustando los preprocesadores y transformando los datos
ohe_cats = ohe.fit_transform(catcols)
mmscaler_num = mmscaler.fit_transform(numcols)

/Users/savelasquez/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/savelasquez/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/savelasquez/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/savelasquez/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future 

In [23]:
ohe.feature_names_in_

array(['FAVC', 'SMOKE', 'CALC', 'MTRANS'], dtype=object)

In [24]:
# Guardando los preprocesadores
joblib.dump(ohe, 'model_assets/ohe.pkl')
joblib.dump(mmscaler, 'model_assets/mmscaler.pkl')

['model_assets/mmscaler.pkl']

In [25]:
ohe_cats

array([[1., 0., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 1., ..., 0., 1., 0.],
       ...,
       [0., 1., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 0.]])

In [26]:
df_processed = np.concatenate([ohe_cats, mmscaler_num], axis = 1)

In [27]:
# Se dividen los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_processed, y, test_size=0.2, random_state=42)

n_estimators = 100 
max_depth = 6
max_features = 4

modelo_random_forest = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
modelo_random_forest.fit(X_train, y_train) 
predictions = modelo_random_forest.predict_proba(X_test)
exactitud = accuracy_score(y_test, modelo_random_forest.predict(X_test))
print(f'Exactitud: {exactitud:.2f}')

/Users/savelasquez/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/savelasquez/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Exactitud: 0.63


/Users/savelasquez/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/savelasquez/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/savelasquez/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/savelasquez/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spa

In [28]:
joblib.dump(modelo_random_forest, 'model_assets/modelo_random_forest.pkl')

['model_assets/modelo_random_forest.pkl']

In [29]:
df

,FAVC,FCVC,NCP,SMOKE,CH2O,TUE,CALC,MTRANS
0,0,2.0,3.0,0,2.000000,1.000000,no,Public_Transportation
1,0,3.0,3.0,1,3.000000,0.000000,Sometimes,Public_Transportation
2,0,2.0,3.0,0,2.000000,1.000000,Frequently,Public_Transportation
3,0,3.0,3.0,0,2.000000,0.000000,Frequently,Walking
4,0,2.0,1.0,0,2.000000,0.000000,Sometimes,Public_Transportation
...,...,...,...,...,...,...,...,...
2106,1,3.0,3.0,0,1.728139,0.906247,Sometimes,Public_Transportation
2107,1,3.0,3.0,0,2.005130,0.599270,Sometimes,Public_Transportation
2108,1,3.0,3.0,0,2.054193,0.646288,Sometimes,Public_Transportation
2109,1,3.0,3.0,0,2.852339,0.586035,Sometimes,Public_Transportation


In [30]:
ohe.feature_names_in_

array(['FAVC', 'SMOKE', 'CALC', 'MTRANS'], dtype=object)